In [ ]:
import pandas as pd
from data_funcs import from_json
import matplotlib.pyplot as plt

In [ ]:
rnn = from_json("rnn_errs.json")
kf = from_json("kf_errs.json")

In [ ]:
# Remove "filename" key
del rnn['filename']
del kf['filename']

In [ ]:
# Prepare lists to store the extracted values
cases = []
predictions = []

# Iterate through the dictionary to extract the needed values
for key, value in rnn.items():
    cases.append(value['case'])
    predictions.append(value['errs']['prediction'])

# Create the DataFrame
df1 = pd.DataFrame({
    'case': cases,
    'prediction': predictions
})


In [ ]:
df1

In [ ]:
df2 = pd.DataFrame.from_dict(kf).transpose()
df2.head()

In [ ]:
plt.hist(df1.prediction, label = "RNN")
plt.hist(df2.errs, label = "KF")
plt.legend()

In [ ]:
print(df1.prediction.mean())
print(df1.prediction.median())

In [ ]:
print(df2.errs.mean())
print(df2.errs.median())